In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import logging
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Import from scripts
sys.path.append("..") 
from src.data_loader import load_tsla_data
#from scripts.arima import train_arima, forecast_arima, evaluate_arima, plot_results,preprocess_data

In [3]:
tesla_data = load_tsla_data('../data/TSLA_cleaned.csv')  



2025-03-01 22:16:37,524 - INFO - Loading TSLA data from CSV...
2025-03-01 22:16:37,622 - INFO - Verifying TSLA data integrity...
2025-03-01 22:16:37,625 - INFO - TSLA data loaded successfully. Shape: (2505, 9), Missing values: 0
